In [ ]:
from logging import DEBUG, INFO
from sdsstools import get_logger
from sdsstools.logger import StreamFormatter

logger = get_logger("astro.net")
logger.sh.setLevel(DEBUG)
logger.sh.formatter = StreamFormatter(fmt='%(asctime)s %(name)s \033[1m%(message)s\033[21m')
logger.debug("hello world")


In [ ]:
import astropy
import astrometry

solver = astrometry.Solver(
    astrometry.series_5200.index_files(
        cache_directory="astrometry_cache",
        scales={6},
    )
)
# only first iteration
logodds_callback = astrometry.Action.STOP
# full
#logodds_callback = astrometry.Action.CONTINUE


In [ ]:
from cluplus.proxy import invoke
from lvmtipo.actors import lvm

telescope_subsystem_name = "sci"
tss = await lvm.from_string(telescope_subsystem_name).start()

await invoke(tss.pwi.status(), tss.km.status())


In [ ]:
from lvmagp.images import Image
from lvmagp.images.processors.detection import DaophotSourceDetection, SepSourceDetection

from plotimg import plot_images

# define source detection
source_detection = SepSourceDetection()
#source_detection = DaophotSourceDetection()

source_count = 17

logger.debug("Expose")
filenames = (await tss.agc.expose(0.5)).flatten().unpack("*.filename") 
images = [Image.from_file(f) for f in filenames]
image = images[0]

plot_images(images)

In [ ]:
arcsec_per_pixel=1/image.header['PIXELSC']*image.header['BINX']
print(f"{image.header['RA']=}")
print(f"{image.header['DEC']=}")
print(f"{arcsec_per_pixel=}")

logger.debug("Source detect")
image = await source_detection(image)
if image.catalog is None:
    logger.warning("No catalog found in image.")
    
logger.debug("Sort")
sources = image.catalog
sources.sort("peak")
sources.reverse()
sources = sources[:source_count]
#logger.debug(f"{sources}")


logger.debug("Solve start")
solution = solver.solve(
    stars_xs=sources['x'],
    stars_ys=sources['y'],
    size_hint=astrometry.SizeHint(
        lower_arcsec_per_pixel=arcsec_per_pixel-0.1,
        upper_arcsec_per_pixel=arcsec_per_pixel+0.1,
    ),
    position_hint=astrometry.PositionHint(
        ra_deg=image.header['RA'],
        dec_deg=image.header['DEC'],
        radius_deg=1,
    ),
    solve_id=None,
    tune_up_logodds_threshold=14.0, # None disables tune-up (SIP distortion)
    output_logodds_threshold=21.0,
    logodds_callback=lambda logodds_list: logodds_callback
)
logger.debug("Solve done")

if solution.has_match():
    print(f"{solution.best_match().center_ra_deg=}")
    print(f"{solution.best_match().center_dec_deg=}")
    print(f"{solution.best_match().scale_arcsec_per_pixel=}")
    wcs = astropy.wcs.WCS(solution.best_match().wcs_fields)
    pixels = wcs.all_world2pix(
        [[star.ra_deg, star.dec_deg] for star in solution.best_match().stars],
        0,
    )
    #print(f"{wcs}")